# OpenMM - An Overview

This notebook provides a practical introduction to the molecular dynamics (MD) engine OpenMM and was prepared for a journal club presentation at the [Structural Bioinformatics and Computational Biochemistry Unit](http://sbcb.bioch.ox.ac.uk/) at the University of Oxford. It is inspired by [OpenMM 7: Rapid development of high performance algorithms for molecular dynamics](https://doi.org/10.1371/journal.pcbi.1005659), the paper introducing the latest version of OpenMM.

### OpenMM - Who is it for?

We already have AMBER, CHARMM, GROMACS, and NAMD (not to mention Expresso, Desmond, LAMMPS, Tinker, and many others), so why do we need yet another MD engine?

The answer is: flexibility! OpenMM provides its users with an unrecedented level of control over their simulations, starting from the ability to write custom simulation scripts in Python to the capability of defining novel forces and even integrators. At the same time, it provides competetive performance by implementing kernels to run simulations on GPU hardware - without requiring the user to write CUDA or even C++ code. These features together make OpenMM a powerful research tool for the molecular simulation community.

### And who should stay away?

There is only one real shortcoming of OpenMM: It does not support MPI parallelism. So if you plan on simulating very large systems that would benefit from scaling to hundrets or thousands of cores, OpenMM might not be the best choice. Similarly, if the hardware you have available does not come with GPUs, you will be limited to a handful of threads and will not see competetive performance.

Another thing to keep in mind is that OpenMM's flexibility comes at the cost of an increased ramp up time. Writing a robust simulation script for productive use takes some time even for experienced Python programmers. If all you want to do is run a simple MD simulation, you might achieve that goal faster by using a tool like GROMACS.

## Installing OpenMM

By far the easiest way of installing OpenMM is through the `conda` package manager of Anaconda Python. This requires no compilation from source, needs no root rights, and takes care of all dependencies except for CUDA.

As of OpenMM 7.3, `conda` packages compatible with all CUDA versions back to 7.5 are available from the [Anaconda Cloud](https://anaconda.org/omnia/openmm). All you need to do is specify the right label when selecting the installation channel.

This is an example of how you would install it:

```bash
# create a new conda environment for OpenMM:
conda create -n openmm-7-3-0-cuda90 python=3.7

# activate the newly created environment:
source activate openmm-7-3-0-cuda90

# install FFT library from omnia channel:
# (needs to be installed separately because of channel conflicts)
conda install -c omnia fftw3f

# install openmm with appropriate CUDA labal:
# (CUDA needs to be installed on the system to run simulations on the GPU)
conda install -c omnia/label/cuda90 openmm=7.3.0

# also install IPython in this environment:
# (provides a neater interactive python shell)
conda install ipython

# to use this environment as kernel in a Jupyter notebook:
conda install ipykernel
```

Once installed, the installation can be tested via:

```bash
python -m simtk.testInstallation
```
OpenMM can now be imported into a Python script like any other Python module.


## Running a Simple Simulation

To run simulations with OpenMM, you need to write a Python script. In this script, you load a starting configuration and force field data and specify the algorithms used to carry out the simulation. The script can then be executed like any other Python script - including from a Jupyter notebook as in this case.

The example below illustrates this on [DHFR](https://en.wikipedia.org/wiki/Dihydrofolate_reductase) in explicit solvent, using the AMBER99SB-ILDN force field and TIP3P water model.

In [2]:
# import OpenMM library and application layer:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

# other imports:
from sys import stdout

# load an input structure and topology data:
print("--> loading input data...")
pdb = PDBFile('../data/dhfr.pdb')
forcefield = ForceField('amber99sbildn.xml', 'tip3p.xml')

# assemble a simulation system:
system = forcefield.createSystem(
    pdb.topology,          # will guess standard residue bonds and use CONECT records for ligands
    nonbondedMethod = PME, # periodic boundaries with PME for Coulomb interactions
    nonbondedCutoff = 1*nanometer,    # Lennard-Jones cutoff
    constraints = HBonds   # SHAKE contraints on bonds with hydrogen
)

# define an integrator:
integrator = VerletIntegrator(0.002*picoseconds)

# decide whether to run on CPU or accelerator hardware:
platform = Platform.getPlatformByName("CPU")

# create a simulation object:
simulation = Simulation(
    pdb.topology, 
    system, 
    integrator,
    platform
)

# initialise positions and velocities:
simulation.context.setPositions(pdb.positions)
simulation.context.setVelocitiesToTemperature(300*kelvin)

# perform 
print("--> running energy minimsation...")
energy_before = simulation.context.getState(getEnergy=True).getPotentialEnergy()
simulation.minimizeEnergy(maxIterations = 100)
energy_after = simulation.context.getState(getEnergy=True).getPotentialEnergy()
print("--> reduced energy from " + str(energy_before) + " to " + str(energy_after))

# will report simulation progress on the command line:
simulation.reporters.append(
    StateDataReporter(
        stdout,     # could also be a file
        reportInterval = 10, 
        time = True, 
        potentialEnergy = True, 
        temperature = True,
        volume = False,
        speed = True,
        separator = "\t"
    )
)

# will also write trajectory data to file:
simulation.reporters.append(
    DCDReporter(
        'output.dcd', 
        reportInterval = 10
    )
)

# perform time integration:
print("--> running time integration...")
simulation.step(100)



--> loading input data...
--> running energy minimsation...
--> reduced energy from -299285.6227214655 kJ/mol to -366434.9033171318 kJ/mol
--> running time integration...
#"Time (ps)"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"
0.020000000000000004	-340040.34812195454	159.56501907873937	0
0.04000000000000002	-338744.1517979552	153.40541840535846	2.75
0.06000000000000004	-338423.55678689375	151.7128078471142	2.75
0.08000000000000006	-337416.8118224936	146.66535026806474	2.77
0.10000000000000007	-336944.2878138131	144.28589539556998	2.73
0.12000000000000009	-336466.0436581623	142.0984739115031	2.72
0.1400000000000001	-337149.0470295342	145.41448838070258	2.76
0.16000000000000011	-337534.0067801288	147.38874373892395	2.73
0.18000000000000013	-337917.93918418715	149.31512748773332	2.72
0.20000000000000015	-338849.20130252454	153.82965818238407	2.73


The example above is relatively simple. In practice, you would usually add a thermostat and barostat to sample from the NPT ensemble. You would likely also run the simulation on the GPU and potentially start not from an PDB structure but from a checkpoint file capturing the state of a previous simulation. Or you might want to start from a configuration and topology generated with a different MD engine (like GROMACS or CHARMM).

All that is possible with OpenMM, but beyond the scope of this simple overview. Take a look at [OpenMM's user manual](http://docs.openmm.org/7.3.0/userguide/index.html) for a detailed description.

## Custom Forces

Custom forces are a unique feature of OpenMM which makes it possible to define new interactions on top of the ones defined by the force field. All you need to do is provide an expression for the contribution of the force to the internal energy of the simulation system. OpenMM will then calculate the energy derivatves (i.e. forces) via automatic differentiation and translate them into machine code for execuation. This works on both the CPU and the GPU, so even though custom forces are specified in Python, they can be as fast as the standard forces implemented in OpenMM's C++ core.

This feature can be extremely useful

 * to restrain complex biochemical systems in a desirable configuration,
 * to model external effects like a transmembrane voltage,
 * or to apply fictitious forces in free energy calculations.
 
Below are two illustrative examples of the first two use cases.

**Note: Custom forces should be added to the simulation system object before creating the simulation object!**

### External Electric Fields

In the following example, a transmembrane voltage is modelled as a constant electric field along the z-axis of the simulation box (implying that the membrane lies in the x/y-plane). This is implemented via OpenMM's `CustomExternalForce`:

In [2]:
# define a new force in terms of the corresponding potential energy:
# (note that z is interpreted as a particle's z-coordinate)
efield_force = CustomExternalForce("-E*z*q")

# the field strength is the same for all particles, so its value can be set as a global parameter:
avogadro_constant = 6.02214076e23
efield_z = 0.1 * joule / coulomb / nanometer / mole * avogadro_constant
efield_force.addGlobalParameter("E", efield_z.value_in_unit(kilojoule_per_mole/elementary_charge/nanometer))

# different atoms carry different charges, so q must be declared a particle-dependent parameter:
efield_force.addPerParticleParameter("q")

# obtain the system's nonbonded force to get access to charges as defined in the force field:
nbforce = [system.getForce(i) for i in range(system.getNumForces()) if isinstance(system.getForce(i), NonbondedForce)][0]

# loop over all atoms in the system:
for atm in pdb.topology.atoms():
    
    # obtain charge of this atom from the force field's non-bonded force:
    charge = nbforce.getParticleParameters(atm.index)[0].in_units_of(coulomb)

    # set the value of the charge parameter:
    # (note that charge must be given in units of elementary charge)
    efield_force.addParticle(atm.index, [charge.value_in_unit(elementary_charge)] )
      
# finally add the electric field force to the simulation system:
print("--> applying electric field E = " + str(efield_z) + " to " + str(efield_force.getNumParticles()) + " particles")
system.addForce(efield_force)

--> applying electric field E = 6.02214076e+22 J/(nm mol C) to 23558 particles


5

While the assumption of constant field strength is commonly applied, it would be easy to make the field depend non-linearly on z (or x or y). OpenMM's custom force mechanism understands all standard algebraic operations (`+ (add), - (subtract), * (multiply), / (divide), and ^ (power)`) alongside a number of more complex functions (`sqrt, exp, log, sin, cos, sec, csc, tan, cot, asin, acos, atan, sinh, cosh, tanh, erf, erfc, min, max, abs, floor, ceil, step, delta, select`). Read the [CustomExternalForce documentation](http://docs.openmm.org/7.3.0/api-c++/generated/OpenMM.CustomExternalForce.html#_CPPv2N6OpenMM19CustomExternalForceE) for more details.

### C-Alpha Positional Restraints

Positional restraints are a common way of preventing large-scale conformational changes. In OpenMM they can be implemented through a `CustomExternalForce` in a straightforward way. The code below applied a harmonic potential to all C-alpha atoms in the simulation system:

In [4]:
    # create a new external force object:
    # (the periodicdistance function calculates a distance under periodic boundary conditions)
    calpha_restraints = CustomExternalForce("k*periodicdistance(x, y, z, x0, y0, z0)^2")
    
    # define all global (i.e. particle-independent) parameters:
    calpha_restraints.addGlobalParameter("k", 1000*kilojoule_per_mole/nanometer**2)
    
    # declare all other parameters to be particle-dependent:
    calpha_restraints.addPerParticleParameter("x0")
    calpha_restraints.addPerParticleParameter("y0")
    calpha_restraints.addPerParticleParameter("z0")

    # apply the restraining force to selected atoms:
    print("--> applying harmonic positional restraints to CA atoms")
    for atm in pdb.topology.atoms():
        # is this a C-alpha atom?
        if atm.name == "CA":
            # define the values of each per-particle parameter:
            calpha_restraints.addParticle(atm.index, pdb.positions[atm.index])

    # finally add the restraining force to the simulation system:
    system.addForce(calpha_restraints)

--> applying harmonic positional restraints to CA atoms


6

The restraining force could easily be extended to apply to e.g. all heavy protein atoms or to all phosphate headgroups of a lipid.

### Other Custom Forces

OpenMM supports much more than just custom external forces. Amongst other things, it supports the following:

 * `CustomBondForce`, `CustomAngleForce`, and `CustomTorsionForce` for custom bonded interactions
 * `CustomNonbondedForce` for used-defined long-ranged interactions
 * `CustomCentroidBondForce` and `CustomCompoundBondForce` for interactions between multiple particles
 * `CustomCVForce` for forces acting on collective variables defined in terms of all particle positions (including an `RMSDForce`)

Take a look at the OpenMM [force documentation](http://docs.openmm.org/7.3.0/api-c++/library.html#id2) for an overview of all available custom forces. The `openmmtools` extension also provides [useful additional forces](https://openmmtools.readthedocs.io/en/latest/forces.html#).

## Custom Integrators

Custom integrators are another feature that makes OpenMM very versatile and that also opens up the black box of how MD calculations are performed. They provide a mechanism for specifying an algorithm to update particle positions and velocities by only specifying Python code.

Below is an example of how a the canonical leapfrog integrator would be implemented through the `CustomIntegrator` formalism:


In [6]:
# create integrator and specify time step:
integrator = CustomIntegrator(2*femtosecond)

# add temporary helper variable variables (for each degree of freedom):
integrator.addPerDofVariable('x0', 0)

# x0 will hold the position at the current time to later compute velocity:
integrator.addComputePerDof('x0', 'x')

# estimate velocity from acceleration via Newton's law:
# (prior to this step, v is the velocity at time t - dt/2)
integrator.addComputePerDof('v', 'v+dt*f/m')

# advance positions:
integrator.addComputePerDof('x', 'x+dt*v')

# adjust updated positions to fulfill constraints (e.g. water geometry, hydrogen covalent bond length):
integrator.addConstrainPositions()

# calculate actual velocity between new and previous position:
integrator.addComputePerDof('v', '(x-x0)/dt')

4

Of course, the leapfrog integrator is already built into OpenMM alongside several other useful algorithms like Langevin integrators or multiple time stepping integrators. But the `CustomIntegrator` class allows the user to implemented completely novel algorithms, for example to combine MD integration with Monte Carlo moves. Note that the `openmmtools` extension already provides many more [advanced integrators](https://openmmtools.readthedocs.io/en/latest/integrators.html) based on the `CustomIntegrator` class.

## Other Notable Features

OpenMM provides a number of other useful features, which are briefly summarised below:

 * Mixed precision mode: Forces are calculated in single precision, but force accumulation and integration are performed in double precision. This ensures high numerical accuracy without a significant performance loss. Mixed precision mode (as well as single precision mode and double precision mode) can be selected at runtime.
 * Input format interoperability: OpenMM can read coordinate and topology files from other MD engines (including AMBER, CHARMM, DESMOND, and GROMACS). This makes it possible to set up simulations with one tool (e.g. pdb2gmx or CHARMM-GUI) and subsequently run them in OpenMM.
 * Polarisable force fields: OpenMM supports both the AMOEBA and CHARMM-Drude polarisable force fields, both of which can be executed on the GPU.

## Where to next?

The following sources may prove useful in further exploring OpenMM:

* [OpenMM User Manual](http://docs.openmm.org/7.0.0/userguide/index.html) - detailed user guide and theory explanation
* [OpenMM Forum](https://simtk.org/plugins/phpBB/indexPhpbb.php?group_id=161&pluginname=phpBB) - get help for using OpenMM, usually very responsive
* [OpenMM Github Repository](https://github.com/pandegroup/openmm) - report bugs and request or contribute new features
* [OpenMM Tools](https://openmmtools.readthedocs.io/en/0.18.1/) - "A batteries-included toolkit for the GPU-accelerated OpenMM molecular simulation engine."
* [YANK](http://getyank.org/latest/) - Python library for alchemical free energy calculations based on OpenMM
* [ParmEd](http://parmed.github.io/ParmEd/html/index.html) - Python library for topology editing, highly interoperable with OpenMM

You can also search for "OpenMM" on GitHub - many third party packages and extensions from the original developers exist.